# Part 0. Data Prepraration

In [1]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset ['train']
validation_dataset = dataset ['validation']
test_dataset = dataset ['test']

c:\Users\JJWX\anaconda3\envs\NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Part 2. Model Training & Evaluation - RNN

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gensim.downloader as api
import numpy as np
import nltk
import random

In [3]:
model = api.load("glove-wiki-gigaword-100")
vocab_size = len(model.index_to_key) + 1
embedding_dim = model.vector_size
word_index = {word: index+1 for index, word in enumerate(model.index_to_key)} # index 0 is reserved for padding
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [4]:
for word, idx in word_index.items():
    if word in model:
        embedding_matrix[idx] = model[word]

In [5]:
def tokenize(text, word_index):
    ls = nltk.word_tokenize(text)
    return [word_index[word] for word in ls if word in word_index]

X_train = [tokenize(text, word_index) for text in train_dataset['text']]
X_val = [tokenize(text, word_index) for text in validation_dataset['text']]
X_test = [tokenize(text, word_index) for text in test_dataset['text']]
max_length = max(len(seq) for seq in X_train)

In [6]:
X_train = pad_sequences(X_train, maxlen=max_length)
X_val = pad_sequences(X_val, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

In [7]:
y_train = np.array(train_dataset['label'])
y_val = np.array(validation_dataset['label'])
y_test = np.array(test_dataset['label'])

Model Training - Grid Search

In [ ]:
from tensorflow.keras.callbacks import Callback
best_accuracy = {}
class CustomCallback(Callback):
    accuracy = 0
    cur_key = ""
    epochs = 0
    optimizer = ""
    batch_size = 0
    best_model = None
    lr = 0
    def on_train_begin(self, logs=None):
        self.accuracy = 0

    def on_train_end(self, logs=None):
        global best_accuracy
        if self.accuracy > best_accuracy.get("accuracy", 0):
            best_accuracy = {
                "accuracy": self.accuracy,
                "epoch": self.epochs,
                "optimizer": self.optimizer,
                "batch_size": self.batch_size,
                "lr": self.lr
            }
            print("Saved best accuracy for current run:", self.accuracy, "at epoch", self.epochs)
            self.best_model.save(filepath="best_model.keras")
        print("Run completed on:")
        print(self.cur_key)
        print("Best accuracy for current run:", self.accuracy, "at epoch", self.epochs)
        print("Training ended")


    
    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs['val_accuracy']
        if val_accuracy > self.accuracy:
            self.accuracy = val_accuracy
            self.epochs = epoch
            self.best_model = self.model

    def set_key(self, optimizer, batch_size, lr):
        self.optimizer = optimizer
        self.batch_size = batch_size
        self.lr = lr
        self.cur_key = f"optimizer: {optimizer}, batch_size: {batch_size}, lr: {lr}"

In [ ]:
from tensorflow.keras.layers import Dropout
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True
    )
    custom_callback = CustomCallback()
    custom_callback.set_key(optimizer, batch_size, lr)
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  
        SimpleRNN(16, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[custom_callback, early_stopping],
        verbose=2
    )
    return model, history

In [ ]:
for batch_size in [16, 32, 64, 128]:
    for lr in [0.005, 0.01, 0.05, 0.1]:
        for optimizer in ['adam', 'sgd', 'rmsprop', 'adagrad']:
            train_model(optimizer, 100, batch_size, lr)

Epoch 1/100
534/534 - 7s - 12ms/step - accuracy: 0.5705 - loss: 0.6765 - val_accuracy: 0.6116 - val_loss: 0.6587
Epoch 2/100
534/534 - 3s - 6ms/step - accuracy: 0.6012 - loss: 0.6633 - val_accuracy: 0.6670 - val_loss: 0.6203
Epoch 3/100
534/534 - 4s - 7ms/step - accuracy: 0.6363 - loss: 0.6449 - val_accuracy: 0.6238 - val_loss: 0.6531
Epoch 4/100
534/534 - 4s - 7ms/step - accuracy: 0.6617 - loss: 0.6169 - val_accuracy: 0.6670 - val_loss: 0.6074
Epoch 5/100
534/534 - 3s - 6ms/step - accuracy: 0.6372 - loss: 0.6423 - val_accuracy: 0.6144 - val_loss: 0.6566
Epoch 6/100
534/534 - 3s - 5ms/step - accuracy: 0.6363 - loss: 0.6402 - val_accuracy: 0.6811 - val_loss: 0.6138
Epoch 7/100


KeyboardInterrupt: 

In [ ]:
best_accuracy

{'accuracy': 0.7016885280609131,
 'epoch': 35,
 'optimizer': 'adagrad',
 'batch_size': 64,
 'lr': 0.01}

#### Best model is trained with Optimizer: adagrad, Batch_size: 64, Learning_rate: 0.01 (Final Hidden State)

In [ ]:
model, history = train_model(best_accuracy['optimizer'], 100, best_accuracy['batch_size'], best_accuracy['lr'])

Epoch 1/100
134/134 - 4s - 28ms/step - accuracy: 0.5043 - loss: 0.7018 - val_accuracy: 0.5403 - val_loss: 0.6903
Epoch 2/100
134/134 - 1s - 10ms/step - accuracy: 0.5335 - loss: 0.6892 - val_accuracy: 0.5647 - val_loss: 0.6850
Epoch 3/100
134/134 - 1s - 10ms/step - accuracy: 0.5518 - loss: 0.6850 - val_accuracy: 0.5713 - val_loss: 0.6821
Epoch 4/100
134/134 - 1s - 9ms/step - accuracy: 0.5612 - loss: 0.6821 - val_accuracy: 0.5713 - val_loss: 0.6798
Epoch 5/100
134/134 - 1s - 9ms/step - accuracy: 0.5645 - loss: 0.6796 - val_accuracy: 0.5694 - val_loss: 0.6777
Epoch 6/100
134/134 - 1s - 9ms/step - accuracy: 0.5689 - loss: 0.6772 - val_accuracy: 0.5638 - val_loss: 0.6757
Epoch 7/100
134/134 - 1s - 8ms/step - accuracy: 0.5755 - loss: 0.6749 - val_accuracy: 0.5666 - val_loss: 0.6738
Epoch 8/100
134/134 - 1s - 8ms/step - accuracy: 0.5817 - loss: 0.6725 - val_accuracy: 0.5713 - val_loss: 0.6718
Epoch 9/100
134/134 - 1s - 7ms/step - accuracy: 0.5862 - loss: 0.6698 - val_accuracy: 0.5788 - val_lo

In [ ]:
best_model = tf.keras.models.load_model("best_model.keras")
accuracy = best_model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy[1] * 100:.2f}%")

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7187 - loss: 0.5703
Test accuracy: 73.92%


#### Mean pooling

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling1D
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_mean.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),
        SimpleRNN(16, return_sequences=True),
        GlobalAveragePooling1D(),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [ ]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
132/134 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4687 - loss: 0.6984
Epoch 1: val_accuracy improved from -inf to 0.48593, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.4689 - loss: 0.6984 - val_accuracy: 0.4859 - val_loss: 0.6943
Epoch 2/100
133/134 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4869 - loss: 0.6949
Epoch 2: val_accuracy improved from 0.48593 to 0.49719, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.4869 - loss: 0.6949 - val_accuracy: 0.4972 - val_loss: 0.6931
Epoch 3/100
132/134 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5029 - loss: 0.6935
Epoch 3: val_accuracy improved from 0.49719 to 0.51595, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.5029 - loss: 0.6935 - val_accuracy: 0.5159 - val_loss: 0.6922
Epoch 4/100
132/134 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5184 - loss: 0.6924
Epoch 4: val_accuracy improved 

In [ ]:
best_model = tf.keras.models.load_model("model_mean.keras")
accuracy = best_model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy[1] * 100:.2f}%")

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6493 - loss: 0.6289
Test accuracy: 71.20%


#### Max pooling

In [ ]:
from tensorflow.keras.layers import GlobalMaxPooling1D
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_max.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False), 
        SimpleRNN(16, return_sequences=True),
        GlobalMaxPooling1D(),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [ ]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
127/134 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5012 - loss: 0.7078
Epoch 1: val_accuracy improved from -inf to 0.53189, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5016 - loss: 0.7073 - val_accuracy: 0.5319 - val_loss: 0.6873
Epoch 2/100
128/134 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5218 - loss: 0.6920
Epoch 2: val_accuracy improved from 0.53189 to 0.55347, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.5222 - loss: 0.6919 - val_accuracy: 0.5535 - val_loss: 0.6841
Epoch 3/100
129/134 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5456 - loss: 0.6887
Epoch 3: val_accuracy improved from 0.55347 to 0.56754, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.5457 - loss: 0.6887 - val_accuracy: 0.5675 - val_loss: 0.6809
Epoch 4/100
133/134 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5539 - loss: 0.6849
Epoch 4: val_accuracy improved from 0

In [ ]:
best_model = tf.keras.models.load_model("model_max.keras")
accuracy = best_model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy[1] * 100:.2f}%")

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7259 - loss: 0.5401
Test accuracy: 0.7354596853256226


#### Dense layer

In [ ]:
from tensorflow.keras.layers import Flatten
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_dense.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),
        SimpleRNN(16, return_sequences=True),
        Flatten(),
        Dense(62, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [ ]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.5358 - loss: 0.6983
Epoch 1: val_accuracy improved from -inf to 0.58349, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 26s 180ms/step - accuracy: 0.5360 - loss: 0.6983 - val_accuracy: 0.5835 - val_loss: 0.6724
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.6117 - loss: 0.6588
Epoch 2: val_accuracy improved from 0.58349 to 0.65009, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 28s 208ms/step - accuracy: 0.6118 - loss: 0.6587 - val_accuracy: 0.6501 - val_loss: 0.6379
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.6471 - loss: 0.6261
Epoch 3: val_accuracy improved from 0.65009 to 0.66886, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 20s 149ms/step - accuracy: 0.6472 - loss: 0.6261 - val_accuracy: 0.6689 - val_loss: 0.6154
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.6783 - loss: 0.6012
Epoch 4: val_accu

In [ ]:
best_model = tf.keras.models.load_model("model_dense.keras")
accuracy = best_model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy[1] * 100:.2f}%")

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7367 - loss: 0.5282
Test accuracy: 0.7373358607292175
